# Quantiles
How do ENSO quantiles change over time?

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import cartopy.crs as ccrs
import numpy as np
import scipy.stats
import seaborn as sns
import xarray as xr
import tqdm
import pathlib
import cmocean
import os
import cartopy.util
import copy

# Import custom modules
import src.utils
from src.XRO import XRO, xcorr

## set plotting specs
sns.set(rc={"axes.facecolor": "white", "axes.grid": False})

## bump up DPI
mpl.rcParams["figure.dpi"] = 100

## get filepaths
DATA_FP = pathlib.Path(os.environ["DATA_FP"])
SAVE_FP = pathlib.Path(os.environ["SAVE_FP"])

## Load data

In [ ]:
## load tropical SST avg
trop_sst = xr.open_dataset(pathlib.Path(DATA_FP, "cesm/trop_sst.nc"))

## Load T,h (total)
Th_total = xr.open_dataset(DATA_FP / "cesm" / "Th.nc")
Th_total = xr.merge([Th_total, trop_sst])

## compute dTdx
Th_total["dTdx"] = Th_total["T_4"] - Th_total["T_3"]

## load ELI
eli = xr.open_dataset(pathlib.Path(DATA_FP, "cesm/eli.nc"))
Th_total = xr.merge([Th_total, eli])

## get windowed data
Th_total = src.utils.get_windowed(Th_total, stride=60)

## seasonal mean
Th_total = Th_total.resample({"time": "QS-DEC"}).mean()

## get anomalies
Th = Th_total - Th_total.mean("member")

## compute quantiles
quant = Th_total.groupby("time.season").quantile(
    q=[0.05, 0.25, 0.5, 0.75, 0.95], dim=["time", "member"]
)
quant = quant.rename({"quantile": "q"})

## compute difference
delta_q = quant - quant.isel(year=0)

## get spread
spread = quant.sel(q=0.95) - quant.sel(q=0.05)
delta_spread = spread - spread.isel(year=0)

## Analysis

### Timeseries

#### Plot quantiles over time

In [ ]:
VARNAME = "T_3"
SEASON = "DJF"

fig, ax = plt.subplots(figsize=(4, 3))

## plot data
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.05), c="b")
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.50), c="gray")
# ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.75), c="gray", ls="--")
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.95), c="r")

ax.plot(
    quant.year,
    delta_spread[VARNAME].sel(season=SEASON),
    ls="--",
    c="k",
)


# ax.plot(delta_q.year, delta_q["T_34"].sel(season=SEASON, q=0.95))

## format
ax.set_xlim([None, 2000])
ax.set_ylim([-0.2, 1])
ax.axhline(0, c="k", lw=0.5)
plt.show()

Plot gradient over time

In [ ]:
VARNAME = "dTdx"
SEASON = "DJF"

fig, ax = plt.subplots(figsize=(4, 3))

## plot data
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.05), c="r")
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.50), c="gray")
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.95), c="b")

ax.plot(
    quant.year,
    delta_spread[VARNAME].sel(season=SEASON),
    ls="--",
    c="k",
)


# ax.plot(delta_q.year, delta_q["T_34"].sel(season=SEASON, q=0.95))

## format
ax.set_xlim([None, 2000])
ax.set_ylim([-0.3, 0.3])
ax.axhline(0, c="k", lw=0.5)
plt.show()

#### Compare to tropical SST

Idea: warm pool (Niño 4) scales with tropical SST?

In [ ]:
VARNAME = "T_4"
SEASON = "DJF"
QS = 12

## get  clim
trop_sst_clim = src.utils.sel_month(Th_total["trop_sst_05"], QS).mean(
    ["time", "member"]
)
# trop_sst_clim = src.utils.sel_month(Th_total["trop_sst_05"],QS).quantile(q=.95,dim=["time","member"])
delta_clim = trop_sst_clim - trop_sst_clim.isel(year=0)

fig, ax = plt.subplots(figsize=(4, 3))

## plot data
ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.95), c="r")
ax.plot(delta_clim.year, delta_clim, c="k")

## format
ax.set_xlim([None, 2000])
ax.set_ylim([-0.2, 1])
ax.axhline(0, c="k", lw=0.5)
plt.show()

Max El Niño scales with horizontal temperature grad?

In [ ]:
dTdx_clim = (
    (Th_total["T_4"] - Th_total["T_3"]).groupby("time.season").mean(["time", "member"])
)
# dTdx_clim = (Th_total["T_4"]-Th_total["T_3"]).groupby("time.season").quantile(q=0.95,dim=["time","member"])
nino_bound = Th.groupby("time.season").quantile(q=0.95, dim=["time", "member"])

In [ ]:
SEASON = "DJF"

fig, ax = plt.subplots(figsize=(4, 3))

## plot data
ax.plot(dTdx_clim.year, dTdx_clim.sel(season=SEASON), c="k")
ax.plot(nino_bound.year, nino_bound["T_3"].sel(season=SEASON), c="r")

## format
ax.set_xlim([None, 2000])
# ax.set_ylim([2, 2.75])
# ax.axhline(0, c="k", lw=0.5)
plt.show()

In [ ]:
VARNAME = "T_4"
SEASON = "DJF"
QS = 12

## get  clim
trop_sst_clim = src.utils.sel_month(Th_total["trop_sst_05"], QS).mean(
    ["time", "member"]
)
trop_sst_05 = src.utils.sel_month(Th_total["trop_sst_05"], QS).quantile(
    q=0.05, dim=["time", "member"]
)
trop_sst_95 = src.utils.sel_month(Th_total["trop_sst_05"], QS).quantile(
    q=0.95, dim=["time", "member"]
)

fig, ax = plt.subplots(figsize=(4, 3))

## plot data
# ax.plot(delta_q.year, delta_q[VARNAME].sel(season=SEASON, q=0.95), c="r")
ax.plot(trop_sst_05.year, trop_sst_05, c="k")
ax.plot(trop_sst_95.year, trop_sst_95, c="k")
ax.plot(quant.year, quant[VARNAME].sel(season=SEASON, q=0.95), c="r")
# ax.plot(delta_clim.year, delta_clim, c="k")

## format
ax.set_xlim([None, 2000])
# ax.set_ylim([-0.2, 3])
# ax.axhline(0, c="k", lw=0.5)
plt.show()

### Plot PDFs over time

In [ ]:
## specify variable
VARNAME = "T_4"

## specify season
QS = 12

## edges for PDF
edges = np.arange(21, 36, 0.5)

## empty array to hold pdfs
pdfs = []

for year in Th_total.year:

    ## get data
    y = src.utils.sel_month(Th_total.sel(year=year), QS)[VARNAME]

    ## compute pdf
    pdfs.append(src.utils.get_empirical_pdf(y, edges=edges)[0])

## put in dataset
pdfs = xr.Dataset(
    data_vars=dict(
        pdf=(("year", "bin_idx"), np.stack(pdfs, axis=0)),
        lb=("bin_idx", edges[:-1]),
        ub=("bin_idx", edges[1:]),
    ),
    coords=dict(year=Th_total.year.values, bin_idx=np.arange(len(edges) - 1)),
)

colors = sns.color_palette("mako")[::3]

fig, ax = plt.subplots(figsize=(5, 4))

# for year, c in zip([1870, 1970], colors):
ax.stairs(pdfs["pdf"].sel(year=1870), edges=edges, fill=True, alpha=0.3)
ax.stairs(pdfs["pdf"].sel(year=1970), edges=edges, lw=2)

ax.set_xlim([None, 31.5])
plt.show()

In [ ]:
## specify variable
VARNAME = "dTdx"

## specify season
QS = 12

## edges for PDF
# edges = np.arange(-4.2,1.0,.2) # dTdx (total)
edges = np.arange(-2, 2.6, 0.2)  # dTdx (anom.)
# edges = np.arange(-5,5.6,.4)

## empty array to hold pdfs
pdfs = []

## empty array to hold skew values
skews = []
stds = []

for year in Th_total.year:

    ## get data
    y = -src.utils.sel_month(Th.sel(year=year), QS)[VARNAME]
    skews.append(scipy.stats.skew(y.values.flatten()))
    stds.append(np.std(y.values.flatten()))

    ## compute pdf
    pdfs.append(src.utils.get_empirical_pdf(y, edges=edges)[0])

## put in dataset
pdfs = xr.Dataset(
    data_vars=dict(
        pdf=(("year", "bin_idx"), np.stack(pdfs, axis=0)),
        skew=("year", np.array(skews)),
        sigma=("year", np.array(stds)),
        lb=("bin_idx", edges[:-1]),
        ub=("bin_idx", edges[1:]),
    ),
    coords=dict(year=Th_total.year.values, bin_idx=np.arange(len(edges) - 1)),
)

colors = sns.color_palette("mako")[::3]

fig, ax = plt.subplots(figsize=(5, 4))

# for year, c in zip([1870, 1970], colors):
ax.stairs(pdfs["pdf"].sel(year=1870), edges=edges, fill=True, alpha=0.3)
ax.stairs(pdfs["pdf"].sel(year=1970), edges=edges, lw=2)

# ax.set_xlim([None, 31.5])
plt.show()

In [ ]:
plt.plot(pdfs.year, pdfs.sigma)

In [ ]:
sel_ = lambda x: src.utils.sel_month(x, QS)
fig, axs = plt.subplots(1, 2, figsize=(5.5, 2.5))
for ax, y in zip(axs, [1870, 1990]):
    ax.scatter(
        sel_(Th["T_4"].sel(year=y)),
        sel_(Th["dTdx"].sel(year=y)),
        s=3,
        alpha=0.5,
    )
src.utils.set_lims(axs)
plt.show()

In [ ]:
sel_ = lambda x: src.utils.sel_month(x, 12)
fig, axs = plt.subplots(1, 2, figsize=(5.5, 2.5))
for ax, y in zip(axs, [1870, 1970]):
    ax.scatter(
        sel_(Th["T_3"].sel(year=y)),
        sel_(Th["eli_05"].sel(year=y)),
        s=3,
        alpha=0.5,
    )
src.utils.set_lims(axs)
plt.show()

## Spatial

In [ ]:
def plot_level(ax, data, level, ls="-", c="w"):
    """plot single level on hovmoller"""
    ax.contour(
        data.longitude,
        data.z_t,
        data,
        levels=[level],
        colors=c,
        linestyles=ls,
        linewidths=1,
    )
    return

### Load data

In [ ]:
forced, _ = src.utils.load_consolidated()
forced = forced[["T", "T_comp", "sst", "sst_comp", "z20", "z20_comp"]]
forced = src.utils.get_windowed(forced).groupby("time.season").mean("time")
forced = src.utils.reconstruct_wrapper(forced)

### Plot subsurface warming

In [ ]:
## specify which period/month to plot
SEASON = "DJF"

## get data
x0 = forced.sel(year=1870, season=SEASON)
x1 = forced.sel(year=1970, season=SEASON)

fig, axs = plt.subplots(1, 3, figsize=(8, 2.5), layout="constrained")

for ax, clim in zip(axs[:2], [x0, x1]):

    ## temperature
    ax.contourf(
        clim.longitude,
        clim.z_t,
        clim["T"],
        cmap="cmo.thermal",
        levels=np.arange(10, 34, 2),
        extend="both",
    )

## plot difference
axs[-1].contourf(
    clim.longitude,
    clim.z_t,
    (x1 - x0)["T"],
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(1, 0.125),
    extend="both",
)

## plot z20
plot_level(ax=axs[0], data=x0["T"], level=22)
plot_level(ax=axs[1], data=x1["T"], level=22, ls="--", c="k")
plot_level(ax=axs[-1], data=x0["T"], level=22)
plot_level(ax=axs[-1], data=x1["T"], level=22, ls="--", c="k")


## label
src.utils.format_subsurf_axs(axs)
for ax in axs:
    ax.axhline(70, c="magenta", ls="--")
    ax.set_xlim([140, 280])


plt.show()

In [ ]:
Tsub = forced["T"].sel(z_t=100, method="nearest").sel(season="DJF")

fig, ax = plt.subplots(figsize=(3, 5))
ax.contourf(
    Tsub.longitude,
    Tsub.year,
    (Tsub - Tsub.isel(year=0)).transpose("year", ...),
    cmap="cmo.balance",
    levels=src.utils.make_cb_range(1, 0.1),
    extend="both",
)

ax.set_ylim([None, 2000])

## Scratch: same, but with anomalies

In [ ]:
## specify variable
VARNAME = "T_34"

## specify season
QS = 9

## edges for PDF
edges = np.arange(-1.5, 10.5, 0.5)

## empty array to hold pdfs
pdfs = []

for year in Th.year:

    ## get data
    y = src.utils.sel_month(Th.sel(year=year), QS)[VARNAME]
    y = y - y.quantile(q=0.05)
    # print(y.min().values.item())
    # print()

    ## compute pdf
    pdfs.append(src.utils.get_empirical_pdf(y, edges=edges)[0])

## put in dataset
pdfs = xr.Dataset(
    data_vars=dict(
        pdf=(("year", "bin_idx"), np.stack(pdfs, axis=0)),
        lb=("bin_idx", edges[:-1]),
        ub=("bin_idx", edges[1:]),
    ),
    coords=dict(year=Th.year.values, bin_idx=np.arange(len(edges) - 1)),
)

colors = sns.color_palette("mako")[::3]

fig, ax = plt.subplots(figsize=(5, 4))

# for year, c in zip([1870, 1970], colors):
ax.stairs(pdfs["pdf"].sel(year=1870), edges=edges, fill=True, alpha=0.3)
ax.stairs(pdfs["pdf"].sel(year=1990), edges=edges, lw=2)

# ax.set_xlim([None, 31.5])
plt.show()